In [0]:
-- Things that we need in Unpaywall but do not want in OpenAlex should be filtered out in the openalex temp view, created below.
-- create or replace temp view openalex as (
--   select id, doi, title, authorships, publication_date, publication_year, abstract, has_abstract, abstract_inverted_index, ids, language, publisher, apc_list, locations_count from openalex.works.openalex_works -- filter out crossref component types
-- )

create or replace table openalex.works.openalex_works_json as (
  SELECT 
    id, 
    to_json(struct(*)) as json_str
  FROM
    openalex.works.openalex_works
)

In [0]:
OPTIMIZE openalex.works.openalex_works_json
ZORDER BY (id);